### 0. Setting

- 드라이브 mount
- 경로 이동
- 라이브러리 및 모듈 설치

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pwd

'/content'

In [ ]:
# 경로 변경 필요할 경우, 수정
%cd ./drive/MyDrive/DACS_공유ver

/content/drive/.shortcut-targets-by-id/1-MPVU9Fze7zOEScw5UloOg3DXNrtrfHF/DACS_공유ver


In [ ]:
!pip install transformers
!pip install tabulate # 시각화 tool

In [ ]:
# 필요한 라이브러리 및 모듈 import

# 기본
import pandas as pd
import numpy as np
import pickle
import time
import csv
import json
import random
import io
import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# Intensity Model
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import ElectraConfig, ElectraTokenizer, ElectraModel

# Summary Model
from transformers import pipeline

# Retrieval Model
from transformers import BertPreTrainedModel, BertModel, BertConfig, BertTokenizer, AutoModel, AutoTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification

from encoder import PolyEncoder, CrossEncoder
from transform_ver2 import SelectionJoinTransform, SelectionSequentialTransform, SelectionConcatTransform
from chat_function import Poly_function, Cross_function

# 시각화
from tabulate import tabulate

### 1. Depression Intensity Model

In [1]:
seed = 42

def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    if not args['no_cuda'] and torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_val)

default_path = os.getcwd()
config_path = os.path.join(default_path, './Korean_ver/model/intensity/config')
model_path = os.path.join(default_path, './Korean_ver/model/intensity/model')
data_path = os.path.join(default_path, './Korean_ver/model/intensity/data')
config_file = "koelectra-base.json"

CONFIG_CLASSES = {
    "koelectra-base": ElectraConfig,
}

TOKENIZER_CLASSES = {
    "koelectra-base": ElectraTokenizer,
}

MODEL_FOR_INTENSITY = {
    "koelectra-base": ElectraModel,
}

NameError: name 'os' is not defined

In [ ]:
with open(os.path.join(config_path, config_file)) as f:
    args = json.load(f)

set_seed(seed)   # seed 값 설정

In [ ]:
config = CONFIG_CLASSES["koelectra-base"].from_pretrained(   # ElectraConfig.from_pretrained
        args['model_name_or_path'],
)

intensity_tokenizer = TOKENIZER_CLASSES[args['model_type']].from_pretrained(   # model_type: koelectra-base, ElectraTokenizer
    args['model_name_or_path'],
    do_lower_case=args['do_lower_case'],
)

intensity_model = MODEL_FOR_INTENSITY[args['model_type']].from_pretrained(
    args['model_name_or_path'],
    config=config
)

args['device'] = "cuda" if torch.cuda.is_available() and not args['no_cuda'] else "cpu"
intensity_model.to(args['device'])

In [ ]:
def convert_sentence(sent_list):   # 사용자 입력 문장 1개 -> 입력 형태 변환
    context2 = None
    batch_encoding = intensity_tokenizer.batch_encode_plus(
        [(sent_list, context2)], max_length=128, padding="max_length", truncation=True)

    features = []
    inputs = {k: batch_encoding[k][0] for k in batch_encoding}
    inputs['label'] = 0
    features.append(inputs)

    input_id = torch.tensor([f['input_ids'] for f in features], dtype=torch.long)
    input_am = torch.tensor([f['attention_mask'] for f in features], dtype=torch.long)
    input_tts = torch.tensor([f['token_type_ids'] for f in features], dtype=torch.long)
    input_lb = torch.tensor([f['label'] for f in features], dtype=torch.long)
    dataset = TensorDataset(input_id, input_am, input_tts, input_lb)
    return dataset

In [ ]:
class ElectraRegressor(nn.Module):
    def __init__(self, config, model):
        super(ElectraRegressor, self).__init__()
        self.model = model
        self.linear = nn.Linear(config.hidden_size, 128)
        self.relu = nn.ReLU()
        self.out = nn.Linear(128, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = outputs.last_hidden_state[:, 0, :]
        # print(f'logits: {len(logits)}, {len(logits[0])}')
        x = self.linear(logits)
        x = self.relu(x)
        score = self.out(x)
        # print(f'score: {score}')
        return score

In [ ]:
def predict_depression(args, model, eval_dataset, global_step=None):
    results = {}
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    criterion = RMSELoss

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(args['device']) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }
            outputs = model(**inputs)
            loss = criterion(outputs.squeeze(), batch[3].type_as(outputs))
        nb_eval_steps += 1
        if preds is None:
            preds = outputs.squeeze()

        else:
            preds = np.append(preds, outputs.squeeze().detach().cpu().numpy(), axis=0)

    # preds = np.argmax(preds, axis=1)
    return preds.item()

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

In [ ]:
model_name = 'emodep_ko_reg.pt'
model_reg = ElectraRegressor(config, intensity_model).to(args['device'])
model_reg.load_state_dict(torch.load(os.path.join(model_path, model_name), map_location=torch.device('cpu')))

# c1_name = 'emodep_kor_a1.pt'
# c2_name = 'emodep_kor_a2.pt'
# c3_name = 'emodep_kor_a3.pt'
# c4_name = 'emodep_kor_a4.pt'
# c5_name = 'emodep_kor_a5.pt'
# c6_name = 'emodep_kor_a6.pt'
# c7_name = 'emodep_kor_a7.pt'
# c8_name = 'emodep_kor_a8.pt'
# c9_name = 'emodep_kor_a9.pt'

# c1_model = ElectraRegressor(config, intensity_model).to(args['device'])
# c1_model.load_state_dict(torch.load(os.path.join(model_path, c1_name), map_location=torch.device('cpu')))

# c2_model = ElectraRegressor(config, intensity_model).to(args['device'])
# c2_model.load_state_dict(torch.load(os.path.join(model_path, c2_name), map_location=torch.device('cpu')))

# c3_model = ElectraRegressor(config, intensity_model).to(args['device'])
# c3_model.load_state_dict(torch.load(os.path.join(model_path, c3_name), map_location=torch.device('cpu')))

# c4_model = ElectraRegressor(config, intensity_model).to(args['device'])
# c4_model.load_state_dict(torch.load(os.path.join(model_path, c4_name), map_location=torch.device('cpu')))

# c5_model = ElectraRegressor(config, intensity_model).to(args['device'])
# c5_model.load_state_dict(torch.load(os.path.join(model_path, c5_name), map_location=torch.device('cpu')))

# c6_model = ElectraRegressor(config, intensity_model).to(args['device'])
# c6_model.load_state_dict(torch.load(os.path.join(model_path, c6_name), map_location=torch.device('cpu')))

# c7_model = ElectraRegressor(config, intensity_model).to(args['device'])
# c7_model.load_state_dict(torch.load(os.path.join(model_path, c7_name), map_location=torch.device('cpu')))

# c8_model = ElectraRegressor(config, intensity_model).to(args['device'])
# c8_model.load_state_dict(torch.load(os.path.join(model_path, c8_name), map_location=torch.device('cpu')))

# c9_model = ElectraRegressor(config, intensity_model).to(args['device'])
# c9_model.load_state_dict(torch.load(os.path.join(model_path, c9_name), map_location=torch.device('cpu')))

In [ ]:
text_input = "죽고싶어"
test_dataset = convert_sentence(text_input)
prediction = predict_depression(args, model_reg, test_dataset)
prediction

# depression_model = [c1_model, c2_model, c3_model, c4_model, c5_model, c6_model, c7_model, c8_model, c9_model, model_reg]

# for idx, model in enumerate(depression_model):

#   prediction = predict_depression(args, model, test_dataset)
#   print(idx+1, ':', prediction)

### 2. Summary Model

In [ ]:
# 2021 훈민정음 한국어 음성, 자연어 인공지능 경진대회 대화요약 부문 알라꿍달라꿍 팀 요약모델

summary_model_name = 'alaggung/bart-rl' # bart-r3f . bart-pretrained
max_length = 64

dialogue = ['요즘 나 너무 우울해', '무슨 일이야?', '시험을 망쳤어', '왜 어려웠어?', '나도 잘 모르겠어, 공부도 많이 했는데',
            '괜찮아, 다음에 또 기회가 있겠지', '다음에 기회가 없으면?', '너무 걱정하지마']

summarizer = pipeline('summarization', model = summary_model_name)
summarization = summarizer("[BOS]"+"[SEP]".join(dialogue)+"[EOS]", max_length=max_length)

# summary sentence
summarization[0]['summary_text']

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/187M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

'시험을 망쳤다고 하자 다음에 기회가 있을 거라고 걱정하지 말라고 한다.'

### 3. Retrieval-based Response Model

In [ ]:
# device 설정
# gpu 이용할 경우
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# cpu 이용할 경우
# device = torch.device('cpu')

print('my device:', device)

my device: cuda:0


In [ ]:
# 데이터 불러오기
print('1. 후보군.csv 파일 불러오기')
data = pd.read_csv('./Korean_ver/data/gpt_candidate.csv', encoding='utf8')

# 필요한 컬럼만 지정해서 불러오기
cand_data = data['candidate']
cand_data.reset_index(drop=True, inplace = True)
cand_data.head()
print(f'후보군으로 사용하는 데이터의 총 개수는 {len(cand_data)} 문장입니다.')
print()

# 사전 계산 후 저장된 후보군 tensor 값 불러오기
print('2. 사전 저장된 후보군 tensor 값 불러오기')
with open('./Korean_ver/data/gpt_candidate_emb.pickle', 'rb') as file:
  final_cand_emb = pickle.load(file)
print(f'{final_cand_emb.shape[1]} 문장 로드 완료!')
print()

# 모델 불러오기(Poly Encoder, Cross Encoder)
print('3. 사용하는 모델 불러오기')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # gpu 사용

# Poly Encoder Model 경로
# Poly Encoder spec : pretrained model = kcbert base / m = 64 / negative sampling = 1:15
# poly_kcbert_urc
PATH_P = './Korean_ver/model/polyencoder/poly_64_pytorch_model.pt'

# Cross Encoder Model 경로
# Cross Encoder spec : pretrained model = kcbert base / m = 0 / negative sampling = 최종 수정한 코드 이용해서 학습한 Encoder
PATH_C = './Korean_ver/model/crossencoder/cross_0_pytorch_model.pt'

# Epitome Re-ranker Model 경로
PATH_E = './Korean_ver/model/epitome/pytorch_model.bin'

# ★★사전 학습 모델이 같아도 따로 설정해주지 않으면 결과 값에 서로 영향 줌★★

# Pretrained bert model for Poly Encoder Model
bert_name = 'beomi/kcbert-base'
bert_config = BertConfig.from_pretrained(bert_name)
bert = AutoModel.from_pretrained(bert_name)

# Pretrained bert model for Cross Encoder Model
rerank_name = 'beomi/kcbert-base'
rerank_config = BertConfig.from_pretrained(rerank_name)
rerank_bert = AutoModel.from_pretrained(rerank_name)

# Pretrained bert model for Epitome Re-ranker Model
epitome_name = 'klue/bert-base'

# Poly Encoder, Cross Encoder Tokenizer
tokenizer = AutoTokenizer.from_pretrained(bert_name)
tokenizer.add_tokens(['\n'], special_tokens = True)

# Epitome Tokenizer
epitome_tokenizer = AutoTokenizer.from_pretrained(epitome_name)

# Input Embedding
context_transform = SelectionJoinTransform(tokenizer = tokenizer, max_len = 300) # kcbert max 값 300
response_transform = SelectionSequentialTransform(tokenizer = tokenizer, max_len = 35)
concat_transform = SelectionConcatTransform(tokenizer = tokenizer, max_len = 300)

# Load Poly Encoder Model
model = PolyEncoder(bert_config, bert = bert, poly_m = 64)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load(PATH_P), strict=False)
model.to(device)
model.device
print(f'Poly Encoer Model = {PATH_P} 로드 완료!.')

# Load Cross Encoder Model
rerank_model = CrossEncoder(rerank_config, bert = rerank_bert)
rerank_model.resize_token_embeddings(len(tokenizer))
rerank_model.load_state_dict(torch.load(PATH_C), strict=False)
rerank_model.to(device)
rerank_model.device
print(f'Cross Encoder Model = {PATH_C} 로드 완료!.')
print()

# Load Epitome Re-ranker Model
epitome_model = AutoModelForSequenceClassification.from_pretrained(epitome_name, num_labels=3, problem_type='multi_label_classification')
epitome_model.load_state_dict(torch.load(PATH_E), strict=False)
epitome_model.to(device)
epitome_model.device
print(f'Epitome Re-ranker Model = {PATH_E} 로드 완료!.')
print()

# Encoder별 함수 불러오기
p_func = Poly_function(model=model, bert=bert, tokenizer=tokenizer, device = device,
                       context_transform = context_transform, response_transform = response_transform)
c_func = Cross_function(model=rerank_model, bert=rerank_bert, tokenizer=tokenizer, device = device,
                        concat_transform = concat_transform)

print('실행 준비 완료!')

1. 후보군.csv 파일 불러오기
후보군으로 사용하는 데이터의 총 개수는 59977 문장입니다.

2. 사전 저장된 후보군 tensor 값 불러오기
59977 문장 로드 완료!

3. 사용하는 모델 불러오기


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Poly Encoer Model = ./Korean_ver/model/polyencoder/poly_64_pytorch_model.pt 로드 완료!.
Cross Encoder Model = ./Korean_ver/model/crossencoder/cross_0_pytorch_model.pt 로드 완료!.



model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epitome Re-ranker Model = ./Korean_ver/model/epitome/pytorch_model.bin 로드 완료!.

실행 준비 완료!


In [ ]:
# cpu 이용할 경우
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)


# 리스트를 문장으로 변경
# 사용 안함
def lisTostr(list):
    new_sent = str()
    for word in list:
        new_sent += word + ' '
    return new_sent


# 필요한 함수 정의
def softmax(a) :
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

def sigmoid(x):
    return 1/(1+np.exp(-x))

def empathy_predict(model, tokenizer, context, response, er_weight, ex_weight):

    model.eval()
    tokenized_sent = tokenizer(
        context,
        response,
        return_tensors = 'pt',
        truncation = True,
        add_special_tokens = True,
        max_length = 64
    )

    tokenized_sent.to(device)

    with torch.no_grad():
        outputs = model(
            input_ids = tokenized_sent['input_ids'],
            attention_mask = tokenized_sent['attention_mask'],
            token_type_ids = tokenized_sent['token_type_ids']
        )

    logits = outputs.logits
    sigmoid = torch.nn.Sigmoid()
    preds = sigmoid(logits.squeeze())

    er_value = preds[0].item() * er_weight
    in_value = preds[1].item()
    ex_value = preds[2].item() * ex_weight
    final_value = er_value + in_value + ex_value

    return final_value

### 4. Execute Chatbot (한국어 DACS)

In [ ]:
def change_filter(past_dep, current_dep):

  global depression_filter

  """
  past_dep: past depression intensity
  current_dep: current depression intensity
  """

  if past_dep <= current_dep: # 과거보다 현재 우울강도가 더 높음
    print("change current filter")
    if depression_filter == 0:
      depression_filter = 1
    elif depression_filter == 1:
      depression_filter = 0

  else: # 과거보다 현재 우울강도가 낮음
    print("keep current filter")
    if depression_filter == 0:
      depression_filter = 0
    elif depression_filter == 1:
      depression_filter =1

  return depression_filter

In [ ]:
# Korean DACS

print("안녕하세요. 우울증 상담 챗봇 DACS 입니다.")
print("우울증 진단 및 고민 상담을 도와드리고 있습니다.")
print("하고 싶은 말이 있다면 편하게 말씀해주세요.")
print()
print("만약, 상담을 끝내고 싶다면 '종료'를 입력해주세요.")
print("'종료'이 외의 말을 입력할 경우, 대화를 끝내기 어려울 수 있습니다.")

print("*"*70)

print()
print('한국어 DACS:')
name = str(input("귀하의 성함을 말씀해주세요:"))
print()
print(f'{name}님 안녕하세요?')
print(f'{name}님 고민이 있으신가요?')
print()

global all_his
global turn
global depression_filter

# save all/every 4 turn conversation
all_his, sub_his = [], []

# save summary history
summary = []

# save depresion score, entire depression score for graph
depression_score, depression_history = [], []

# turn_history
turn_history = []

# default_setting
turn = 1
er_weight = 1
ex_weight = 1
depression_filter = 0

while True:

    # extract top 10 candidates from Poly Encoder
    idx_list = []
    top_cand = []

    print('내담자: ')
    user_context = [str(input())]
    input_text = convert_sentence(user_context[0])
    # print('input_text:', input_text)
    current_depression = predict_depression(args, model_reg, input_text) # current depression score
    print('current_depression:', current_depression)

    depression_score.append(current_depression)
    depression_history.append(current_depression)

    word = user_context[0].split(' ') # input sentence as token

    # end conversation
    if user_context == ['종료']:
       print("대화를 종료합니다!")
       break
      # 대화 데이터 자동 저장 기능
      #  with open('./test_data/kor_test_data/test_example.csv', 'a', newline = '') as file:
      #   writer = csv.writer(file)
      #   writer.writerow(all_his)
      #   file.close()
      # break

    else:
        all_his.append(user_context[0])
        sub_his.append(user_context[0])

        start = time.time()
        if turn == 1:
          user_emb = p_func.ctx_emb(*p_func.input_context(user_context))

        elif turn <= 4:
          user_emb = p_func.ctx_emb(*p_func.input_context(all_his)) # for multi-turn

        else:
          user_input = [summary[-1]] + user_context # summary 문장 + user_context
          # print(user_input)
          user_emb = p_func.ctx_emb(*p_func.input_context(user_input))

        final_score = p_func.score(user_emb, final_cand_emb)
        new_score = final_score.sort()

        # most relevant 10
        for i in range(10):
          idx_list.append(int(new_score[1][0][-10:][i]))
        for idx in reversed(idx_list):
          top_cand.append(cand_data[idx])

        # save top 10 candidates
        top_cand = pd.Series(top_cand)

        # 1. Reranking model - Cross Encoder for Context
        rerank_list, rerank_context_list, context_score = [], [], []

        if turn == 1:
          cross_score = c_func.text_emb(*c_func.input_text(user_context, top_cand))
          prob_cross_score = F.softmax(cross_score, dim = 0) # sum of probability = 1

        elif turn <= 4:
          cross_score = c_func.text_emb(*c_func.input_text(all_his, top_cand))
          prob_cross_score = F.softmax(cross_score, dim = 0)

        else:
          cross_score = c_func.text_emb(*c_func.input_text(user_input, top_cand))
          prob_cross_score = F.softmax(cross_score, dim = 0)

        for i in range(len(prob_cross_score)):
          rerank_list.append(prob_cross_score[i].item())

        # sorting
        arr_rerank = np.array(rerank_list)
        final_rerank = sigmoid(arr_rerank)

        reversed_idxes = np.argsort(rerank_list)[::-1]

        for i in range(len(prob_cross_score)):
          rerank_context_list.append(top_cand[reversed_idxes[i]])
          context_score.append(final_rerank[reversed_idxes[i]])

        df_cross_cand=pd.DataFrame()
        df_cross_cand['candidate']=rerank_context_list
        df_cross_cand['context_score']=context_score

        # 2. Reranking model - Epitome Reranker for empathy
        empathy_score = []

        # early conversation → questioning
        if turn <= 4:
          ex_weight = 1
          er_weight = 1

        elif depression_filter == 0: # question
          ex_weight = 1
          er_weight = 1

        elif depression_filter == 1: # emotional reaction
          ex_weight = 0.5
          er_weight = 2

        for i, response in enumerate(top_cand):
          score = empathy_predict(epitome_model, epitome_tokenizer, user_context[0], response, ex_weight, er_weight)
          empathy_score.append(score)

        df_epitome_cand=pd.DataFrame()
        df_epitome_cand['candidate']=top_cand
        df_epitome_cand['empathy_score']=empathy_score
        df_epitome_cand

        new_empathy_score=[]

        for i in range(len(df_cross_cand)):
          for j in range(len(df_epitome_cand)):
            if df_cross_cand['candidate'][i] == df_epitome_cand['candidate'][j]:
              new_empathy_score.append(df_epitome_cand['empathy_score'][j])

        df_cross_cand['empathy_score'] = new_empathy_score
        df_cross_cand['sum'] = df_cross_cand['context_score'] + df_epitome_cand['empathy_score']
        df_cross_cand = df_cross_cand.sort_values(by=['sum'], ascending = False)
        df_cross_cand.reset_index(drop=True, inplace=True)

        print(tabulate(df_cross_cand, headers='keys', tablefmt='psql', showindex=True))

        # 중복된 응답을 피하기 위해 세 개 답변을 비교하여 출력

        # select the best response
        first_response = df_cross_cand['candidate'][0]
        # select the second best response
        second_response = df_cross_cand['candidate'][1]
        # select the third best response
        third_response = df_cross_cand['candidate'][2]

        # select the response which is not in conv_his
        if first_response in all_his and second_response in all_his:
            best_response = third_response
        elif turn > 0 and first_response in all_his:
            best_response = second_response
        else:
            best_response = first_response

        print()
        print('한국어 DACS: ')
        print(best_response)

        # save the conversation
        all_his.append(best_response)
        sub_his.append(best_response)

        # sub conversation reset
        if turn % 4 == 0:
          summarization = summarizer("[BOS]"+"[SEP]".join(sub_his)+"[EOS]", max_length=max_length)
          conv_summary = summarization[0]['summary_text']
          print('summary:', conv_summary)
          summary.append(conv_summary)
          sub_his = []

          # check user's preference
          avg_depression_score = sum(depression_score) / len(depression_score)
          print('avg depression:', avg_depression_score)
          print('current depression:', current_depression)

          change_filter(avg_depression_score, current_depression)

          # 4 turn 단위로 초기화
          depression_score = []

        turn_history.append(turn)
        turn += 1


        end = time.time()
        # print('To reply',round(end-start,2),'sec is consumed')
        print()

안녕하세요. 우울증 상담 챗봇 DACS 입니다.
우울증 진단 및 고민 상담을 도와드리고 있습니다.
하고 싶은 말이 있다면 편하게 말씀해주세요.

만약, 상담을 끝내고 싶다면 '종료'를 입력해주세요.
'종료'이 외의 말을 입력할 경우, 대화를 끝내기 어려울 수 있습니다.
**********************************************************************

한국어 DACS:
귀하의 성함을 말씀해주세요:김미래

김미래님 안녕하세요?
김미래님 고민이 있으신가요?

내담자: 
자도 자도 피곤해요
current_depression: 6.458701133728027
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+-----------------+---------+
|    | candidate                                                                                                                                                                                                |   context_score |   empathy_score |     sum |
|----+-------------------------------------------------------------------------------------------------------------------------------------------------